In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import random

from qiskit import QuantumCircuit, Aer, execute
from qiskit.quantum_info import Operator
from scipy.linalg import expm

from src.Tools import get_qiskit_H
from src.Tools import get_full_hamiltonian

from src.CPQAOA import CP_QAOA
from src.QAOA import QAOA
from src.Chain import Chain
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       normalized_cost, 
                       qubo_limits, 
                       check_qubo)

In [2]:
# Number of Qubits 
N=6
# number of excitations
k=3
# RNG seed for reproducibility
seed=1
# alpha in: s^T*mu + alpha*(s^T*Covar*s)
alpha=0.5
# Nr. of layer repetitions
layers=1
# Maximal number of iterations for classical solver
max_iter=500

my_chain = Chain(N_qubits=N)
print(f' Qubit indices in chain: \n', my_chain.get_chain_indexing())
print(f'\n Corresponding Nearest Neighbor index pairs: \n', my_chain.get_NN_indices())
print(f'\n Corresponding Nearest Neighbor + Next Nearest index pairs: \n', my_chain.get_NNN_indices())
init_strat = np.array([0,1,0,1,0,1])
my_chain.set_initialization_strategy(strategy=init_strat)
print(f'\n Initialization strategy is: \n', my_chain.get_initialization_strategy())
print(f'\n Corresponding indices is: \n', my_chain.get_initialization_indices())

# Deciding between grid and 1d chain topology
my_topology = my_chain

 Qubit indices in chain: 
 [0 1 2 3 4 5]

 Corresponding Nearest Neighbor index pairs: 
 [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5)]

 Corresponding Nearest Neighbor + Next Nearest index pairs: 
 [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (3, 5), (4, 5)]

 Initialization strategy is: 
 [0 1 0 1 0 1]

 Corresponding indices is: 
 [1, 3, 5]


In [5]:
# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=N, seed=seed)

# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
full_space_max_cost = full_result['c_max']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])
print(f"Min. cost portfolio (constrained subspace): {portfolio_subspace_min_cost}")
print("Optimal portfolio state (constrained subspace) is: |"+portfolio_subspace_min_state_str+">")

# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])
print(f"Min. cost QUBO: {qubo_min_cost}")
print("Min. cost QUBO state is: |"+qubo_min_state_str+">")
print("Check that qubo min cost is same as portfolio min cost:")
print(qubo_min_cost, portfolio_subspace_min_cost)
print("Check that qubo max cost is at least portfolio max cost:")
print(qubo_max_cost, portfolio_subspace_max_cost)


Min. cost portfolio (constrained subspace): 4.931996196739781
Optimal portfolio state (constrained subspace) is: |110010>
Min. cost QUBO: 4.931996196739796
Min. cost QUBO state is: |110010>
Check that qubo min cost is same as portfolio min cost:
4.931996196739796 4.931996196739781
Check that qubo max cost is at least portfolio max cost:
71.68016228241302 8.710003549382588


In [ ]:
initialization_strategy = my_topology.initialization_strategy
approximate_hamiltonian = True
with_z_phase = False
simulator = Aer.get_backend('statevector_simulator')
qubit_indices = {0:my_topology.get_NN_indices(),
                 1: [],
                 2: [],
                 3: []}
counts = None

def set_circuit(angles):
    __angles__ = iter(angles)

    # Defining circuit
    qcircuit = QuantumCircuit(N)

    # Setting 'k' qubits to |1>
    for qubit_index in initialization_strategy:
        qcircuit.x(qubit_index)

    for layer in range(layers):
        if approximate_hamiltonian:
            # XX+YY terms
            for (qubit_i, qubit_j) in qubit_indices[layer]:
                theta_ij = next(__angles__)
                qcircuit.rxx(theta=theta_ij, qubit1=qubit_i, qubit2=qubit_j)
                qcircuit.ryy(theta=theta_ij, qubit1=qubit_i, qubit2=qubit_j)
            # Z terms
            if with_z_phase:
                for qubit_i in range(N):
                    qcircuit.rz(phi=next(__angles__), qubit=qubit_i)
        else:
            H = get_full_hamiltonian(indices=qubit_indices[layer],
                                     angles=angles[layer*len(angles)//layers:(layer+1)*len(angles)//layers],
                                     N_qubits=N,
                                     with_z_phase=with_z_phase)
            time = 1.0
            U_H = Operator(expm(-1j*time*H.data))
            qcircuit.append(U_H, list(range(N)))
    return qcircuit

def get_cost(angles) -> float:
    circuit = set_circuit(angles=angles)
    counts = execute(circuit, simulator).result().get_counts()
    H_c = np.array(Operator(get_qiskit_H(Q=Q)))
    state_vector = np.array(execute(circuit, simulator).result().get_statevector()).flatten()
    return float(np.real(np.dot(state_vector.conj(), np.dot(H_c, state_vector))))